In [1]:
import pandas as pd
import numpy as np

Must be at least 26 prior points of data to calculate all technical indicators that will be used as features at a certain point

In [43]:
df = pd.read_csv('PBA.csv')
data = np.array(df)
print(data)

[[3.2190000e+01 3.2230000e+01 3.1640000e+01 3.2100000e+01           nan
  1.6251840e+09 8.8423900e+05 3.1480000e+01]
 [3.1740000e+01 3.2210000e+01 3.1340000e+01 3.2210000e+01           nan
  1.6255296e+09 7.1682900e+05 3.1140000e+01]
 [3.1330000e+01 3.1980000e+01 3.1210000e+01 3.1670000e+01           nan
  1.6256160e+09 6.7474200e+05 3.0760000e+01]
 [3.1130000e+01 3.1440000e+01 3.0610000e+01 3.0930000e+01           nan
  1.6257024e+09 7.2232300e+05 3.0500000e+01]
 [3.2070000e+01 3.2180000e+01 3.1420000e+01 3.1500000e+01           nan
  1.6257888e+09 7.6691700e+05 3.0880000e+01]
 [3.2230000e+01 3.2300000e+01 3.1600000e+01 3.1870000e+01           nan
  1.6260480e+09 7.4633000e+05 3.1740000e+01]
 [3.2100000e+01 3.2210000e+01 3.1760000e+01 3.2120000e+01           nan
  1.6261344e+09 7.6633200e+05 3.1390000e+01]
 [3.1480000e+01 3.2280000e+01 3.1390000e+01 3.2280000e+01           nan
  1.6262208e+09 7.0624100e+05 3.1360000e+01]
 [3.1140000e+01 3.1540000e+01 3.1030000e+01 3.1250000e+01       

# Relative Strength Index (RSI)

RSI is only calculated on the last 14 values

In [29]:
#calculating RSI
def RSI(data):
    gainavg = 0
    lossavg = 0
    for i in range(len(data)-14,len(data)):
        #difference of yesterdays and todays closing price
        #absolute value of losses (negative numbers when subtracting yesterdays closing from todays closing) is taken
        gain_loss = data[i][0]-data[i-1][0]
        if(gain_loss<=0):
            lossavg += gain_loss*-1
        else:
            gainavg += gain_loss
    gainavg/=14
    lossavg/=14
    #actual RSI
    return 100 - (100/(1+(gainavg/lossavg)))

In [30]:
RSI(data)

61.061946902654846

# Money Flow Index (MFI)

Last 14 values

In [5]:
#calculating MFI
def MFI(data):
    count = 0
    pos_flow = 0
    neg_flow = 0
    for i in range(len(data)-14,len(data)):
        #money flow = (close+low+high)/3 * volume
        #determine if positive or negative money flow by whether the flow has gone up from the previous day or down
        current_flow = ((data[i][0]+data[i][1]+data[i][2])/3) * data[i][len(data[i])-1]
        prev_flow = ((data[i-1][0]+data[i-1][1]+data[i-1][2])/3) * data[i-1][len(data[i-1])-1]
        if(current_flow-prev_flow <= 0):
            neg_flow += current_flow
        else:
            pos_flow += current_flow
    #actual MFI
    return 100 - (100/(1+(pos_flow/neg_flow)))

In [6]:
MFI(data)

15.45830332582679

# Exponential Moving Average (EMA)

Dynamic time periods, using 5 day EMAs to calculate longer period EMA

In [7]:
#calculating EMA
def EMA(data, period):
    #calculate SMA for a 5 day period first
    SMA = 0
    for i in range(len(data)-period, len(data)-period+5):
        SMA += data[i][0]
    SMA /= 5
    #smoothing weight
    sw = 2 / (5+1)
    #calculate EMA for each 5 day period moving forward through the latest 14 points of data
    prevEMA = SMA
    for i in range(len(data)-period+6, len(data)):
        EMA = data[i][0]*sw
        EMA += prevEMA * (1 - sw)
        prevEMA = EMA
    return EMA

In [8]:
EMA(data,14)

30.565270233196163

# Stochastic Oscillator

14 day again

In [9]:
#calculationg Stochastic Oscillator
def SO(data):
    low = data[len(data)-15][0]
    high = data[len(data)-15][0]
    for i in range(len(data)-15, len(data)-1):
        if data[i][0] < low:
            low = data[i][0]
        if data[i][0] > high:
            high = data[i][0]
    return ((data[len(data)-1][0] - low)/(high-low))*100

In [10]:
SO(data)

-13.711583924349924

# Moving Average Convergence Divergence (MACD)

12 day EMA - 26 day EMA

In [11]:
#calculating MACD
def MACD(data):
    return EMA(data,12) - EMA(data,26)

In [12]:
MACD(data)

0.005826539151136956

In [48]:
f = open('PBAindicators.csv','w')
f.write('RSI,MFI,EMA,SO,MACD,Y\n')
for i in range(26,len(data)):
    rsi = RSI(data[i-15:i])
    print(f'RSI:{rsi}')
    mfi = MFI(data[i-15:i])
    print(f'MFI:{mfi}')
    ema = EMA(data[i-15:i],14)
    print(f'EMA:{ema}')
    so = SO(data[i-15:i])
    print(f'SO:{so}')
    macd = MACD(data[i-26:i])
    print(f'MACD:{macd}')
    y = data[i][len(data[i])-1]
    print(f'y value:{y}')
    print('\n\n')
    f.write(str(rsi) + "," + str(mfi) + "," + str(ema) + "," + str(so) + "," + str(macd) + "," + str(y))
    f.write("\n")
f.close()

RSI:56.28930817610065
MFI:43.245438201266836
EMA:32.170284407864656
SO:33.472803347280305
MACD:-0.02081713422234799
y value:30.03



RSI:53.82059800664455
MFI:36.292435802840366
EMA:32.17051303155007
SO:43.979057591623125
MACD:0.019404116635541868
y value:30.0



RSI:59.02192242833053
MFI:43.13016266693444
EMA:32.26966194177717
SO:57.591623036649175
MACD:0.005869052242559292
y value:30.88



RSI:58.33333333333333
MFI:43.011184092578574
EMA:32.31454961133974
SO:52.35602094240828
MACD:0.01776991003580264
y value:31.08



RSI:45.806451612903295
MFI:42.81042558137368
EMA:32.31991159884165
SO:41.50943396226429
MACD:0.017594318491958916
y value:31.06



RSI:45.90517241379316
MFI:42.986968591472376
EMA:32.229465325407716
SO:22.012578616352243
MACD:0.021978488616049674
y value:30.45



RSI:38.37953091684431
MFI:43.227963494518654
EMA:32.03891144642586
SO:-1.8867924528302562
MACD:0.026571132752977178
y value:31.11



RSI:25.24850894632202
MFI:42.890525701015555
EMA:31.62385246151502
SO:-53.7037